In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
comex_df=pd.read_csv('/home/andre301267/git/Pricing-Fertilizer/DB/temp/imp_fert_ncm_quanti.csv')

In [2]:
mes_pred=10
comex_df=comex_df[(comex_df.Product.isin(['MOP']))&(comex_df.UF.isin(['BA']))&(comex_df.month.isin([mes_pred]))]

In [3]:
filtra_km=comex_df[['Product','km']].drop_duplicates().sort_values(by=['Product','km'],ascending=False).drop_duplicates(subset='Product')
comex_df=comex_df.merge(filtra_km)

In [4]:
comex_df['date']=pd.to_datetime(comex_df.year.astype(str)+'-'+comex_df.month.astype(str)+'-01')
date_min=comex_df.date.min()
date_max=comex_df.date.max()

In [5]:
years=pd.DataFrame({'year':np.arange(comex_df.year.min(),comex_df.year.max()+1)}).assign(key=0)
months=pd.DataFrame({'month':[mes_pred]}).assign(key=0)
calendar=years.merge(months,on='key')
full=comex_df[['Product','UF']].drop_duplicates().assign(key=0).merge(calendar,on='key').drop('key',axis=1)
comex_df=comex_df.merge(full)

In [6]:
round(comex_df[comex_df.Product=='MOP'].kg.sum()/10**9)

1

In [7]:
comex_df=comex_df.groupby(by=['year','month','Product'],observed=False).kg.sum().reset_index()

In [8]:
comex_df['date']=pd.to_datetime(comex_df.year.astype(str)+'-'+comex_df.month.astype(str)+'-01')

In [9]:
comex_df=comex_df[(comex_df.date>=date_min)&(comex_df.date<=date_max)]

In [10]:
comex_df[['month','Product']]=comex_df[['month','Product']].astype(str)

In [11]:
comex_df=pd.get_dummies(comex_df)

In [12]:
comex_df=comex_df.assign(oc=0)
comex_df.loc[comex_df.kg>0,'oc']=1

# Classify

In [13]:
comex_df.shape

(26, 6)

In [14]:
comex_df

,year,kg,date,month_10,Product_MOP,oc
0,1997,1.724462e+07,1997-10-01,True,True,1
1,1998,9.150000e+06,1998-10-01,True,True,1
2,1999,1.050000e+07,1999-10-01,True,True,1
3,2000,1.872611e+07,2000-10-01,True,True,1
4,2001,1.044524e+07,2001-10-01,True,True,1
5,2003,1.161144e+07,2003-10-01,True,True,1
6,2004,1.762914e+07,2004-10-01,True,True,1
7,2005,9.974000e+06,2005-10-01,True,True,1
8,2006,3.671255e+07,2006-10-01,True,True,1
9,2007,2.180257e+07,2007-10-01,True,True,1


In [15]:
comex_df_t=comex_df.copy()
for r in range(1,6):
    train=comex_df_t[comex_df_t.year<comex_df_t.year.max()]
    test=comex_df_t[comex_df_t.year==comex_df_t.year.max()]
    X_train=train.drop(['kg','oc','date'],axis=1)
    y_train=train.oc
    X_test=test.drop(['kg','oc','date'],axis=1)
    y_test=test.oc
    from sklearn.ensemble import RandomForestClassifier as rfc
    model=rfc()
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    test['y_pred']=y_pred
    test['y_test']=y_test
    if r==1:
        results=test
    else:
        results=pd.concat([results,test])
    comex_df_t=comex_df_t[comex_df_t.year<comex_df_t.year.max()]

In [16]:
comex_df=results.merge(comex_df,how='outer')

In [17]:
comex_df.shape

(26, 9)

In [18]:
comex_df=comex_df[(comex_df.oc==1)&(comex_df.y_pred.isin([1,np.nan]))]

In [19]:
comex_df.shape

(26, 9)

# Regression

In [20]:
comex_df_t_t=comex_df.copy()
features_df=pd.DataFrame()
for r in range(1,11):
    train=comex_df_t_t[comex_df_t_t.year<comex_df_t_t.year.max()]
    test=comex_df_t_t[comex_df_t_t.year==comex_df_t_t.year.max()]
    X_train=train.drop(['kg','oc','date'],axis=1)
    y_train=train.kg
    X_test=test.drop(['kg','oc','date'],axis=1)
    y_test=test.kg
    from sklearn.ensemble import RandomForestRegressor as rfr
    model=rfr(max_features=5)
    model.fit(X_train,y_train)
    t=pd.DataFrame({'names':model.feature_names_in_,'importances':model.feature_importances_})
    features_df=pd.concat([features_df,t])
    y_pred=model.predict(X_test)
    test['y_pred']=y_pred
    test['y_test']=y_test
    test['r']=r
    if r==1:
        results=test
    else:
        results=pd.concat([results,test])
    
    comex_df_t=comex_df_t[comex_df_t.year<comex_df_t.year.max()]


display(features_df.groupby('names').importances.mean().reset_index().sort_values(by='importances',ascending=False))

,names,importances
5,year,0.973213
2,r,0.026787
1,month_10,0.000000
0,Product_MOP,0.000000
3,y_pred,0.000000
4,y_test,0.000000


In [21]:
results=results[results.Product_MOP==1]

In [22]:
results[['date','kg','y_test','y_pred']]

,date,kg,y_test,y_pred
25,2023-10-01,37437366.0,37437366.0,3.330771e+07
25,2023-10-01,37437366.0,37437366.0,3.283028e+07
25,2023-10-01,37437366.0,37437366.0,3.253258e+07
25,2023-10-01,37437366.0,37437366.0,3.277206e+07
25,2023-10-01,37437366.0,37437366.0,3.279754e+07
25,2023-10-01,37437366.0,37437366.0,3.296760e+07
25,2023-10-01,37437366.0,37437366.0,3.220287e+07
25,2023-10-01,37437366.0,37437366.0,3.235381e+07
25,2023-10-01,37437366.0,37437366.0,3.314202e+07
25,2023-10-01,37437366.0,37437366.0,3.366898e+07


In [23]:
from sklearn.metrics import root_mean_squared_error as rse
results['rse']=rse(results.y_test,results.y_pred)

In [24]:
int(results.rse.unique())/10**6

4.598925